In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import numpy as np
import pandas as pd
import os
import pickle

import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
pd.options.display.precision = 15

import lightgbm as lgb
import xgboost as xgb
import time
import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
from sklearn import metrics
from sklearn import linear_model
import gc
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

import eli5
import shap
from IPython.display import HTML
import json
import altair as alt

import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline
import multiprocessing
import warnings
warnings.simplefilter('ignore')
alt.renderers.enable('notebook')

RendererRegistry.enable('notebook')

In [5]:
from functions import save_features2, LabelEncoderPopularity,load_data

In [6]:
folder_path = './data/'
files = [f'{folder_path}train_identity.csv', 
         f'{folder_path}train_transaction.csv',
         f'{folder_path}test_identity.csv',
         f'{folder_path}test_transaction.csv',
         f'{folder_path}sample_submission.csv']

In [7]:
# %%time
train_identity, train_transaction, test_identity, test_transaction, sub = map(load_data, files)

In [8]:
test_transaction['isFraud'] = 'test'
data = pd.concat([train_transaction, test_transaction], axis=0, sort=False )
data.set_index('TransactionID',inplace=True)
del train_transaction
del test_transaction

In [9]:
data_id = pd.concat([train_identity, test_identity], axis=0, sort=False )
# data_id = data.reset_index()
data_id.set_index('TransactionID',inplace=True)
del train_identity
del test_identity


In [10]:
useful_features = ['TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'dist1',
                   'P_emaildomain', 'R_emaildomain', 'C1', 'C2', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13',
                   'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'M2', 'M3',
                   'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V17',
                   'V19', 'V20', 'V29', 'V30', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V40', 'V44', 'V45', 'V46', 'V47', 'V48',
                   'V49', 'V51', 'V52', 'V53', 'V54', 'V56', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V69', 'V70', 'V71',
                   'V72', 'V73', 'V74', 'V75', 'V76', 'V78', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V87', 'V90', 'V91', 'V92',
                   'V93', 'V94', 'V95', 'V96', 'V97', 'V99', 'V100', 'V126', 'V127', 'V128', 'V130', 'V131', 'V138', 'V139', 'V140',
                   'V143', 'V145', 'V146', 'V147', 'V149', 'V150', 'V151', 'V152', 'V154', 'V156', 'V158', 'V159', 'V160', 'V161',
                   'V162', 'V163', 'V164', 'V165', 'V166', 'V167', 'V169', 'V170', 'V171', 'V172', 'V173', 'V175', 'V176', 'V177',
                   'V178', 'V180', 'V182', 'V184', 'V187', 'V188', 'V189', 'V195', 'V197', 'V200', 'V201', 'V202', 'V203', 'V204',
                   'V205', 'V206', 'V207', 'V208', 'V209', 'V210', 'V212', 'V213', 'V214', 'V215', 'V216', 'V217', 'V219', 'V220',
                   'V221', 'V222', 'V223', 'V224', 'V225', 'V226', 'V227', 'V228', 'V229', 'V231', 'V233', 'V234', 'V238', 'V239',
                   'V242', 'V243', 'V244', 'V245', 'V246', 'V247', 'V249', 'V251', 'V253', 'V256', 'V257', 'V258', 'V259', 'V261',
                   'V262', 'V263', 'V264', 'V265', 'V266', 'V267', 'V268', 'V270', 'V271', 'V272', 'V273', 'V274', 'V275', 'V276',
                   'V277', 'V278', 'V279', 'V280', 'V282', 'V283', 'V285', 'V287', 'V288', 'V289', 'V291', 'V292', 'V294', 'V303',
                   'V304', 'V306', 'V307', 'V308', 'V310', 'V312', 'V313', 'V314', 'V315', 'V317', 'V322', 'V323', 'V324', 'V326',
                   'V329', 'V331', 'V332', 'V333', 'V335', 'V336', 'V338', 'id_01', 'id_02', 'id_03', 'id_05', 'id_06', 'id_09',
                   'id_11', 'id_12', 'id_13', 'id_14', 'id_15', 'id_17', 'id_19', 'id_20', 'id_30', 'id_31', 'id_32', 'id_33',
                   'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo']

In [11]:
cols_to_drop = [col for col in data.columns if col not in useful_features]
cols_to_drop.remove('isFraud')
# cols_to_drop.remove('TransactionID')
cols_to_drop.remove('TransactionDT')

In [12]:
cols_to_drop

['dist2',
 'C3',
 'D7',
 'M1',
 'V1',
 'V2',
 'V14',
 'V15',
 'V16',
 'V18',
 'V21',
 'V22',
 'V23',
 'V24',
 'V25',
 'V26',
 'V27',
 'V28',
 'V31',
 'V32',
 'V39',
 'V41',
 'V42',
 'V43',
 'V50',
 'V55',
 'V57',
 'V65',
 'V66',
 'V67',
 'V68',
 'V77',
 'V79',
 'V86',
 'V88',
 'V89',
 'V98',
 'V101',
 'V102',
 'V103',
 'V104',
 'V105',
 'V106',
 'V107',
 'V108',
 'V109',
 'V110',
 'V111',
 'V112',
 'V113',
 'V114',
 'V115',
 'V116',
 'V117',
 'V118',
 'V119',
 'V120',
 'V121',
 'V122',
 'V123',
 'V124',
 'V125',
 'V129',
 'V132',
 'V133',
 'V134',
 'V135',
 'V136',
 'V137',
 'V141',
 'V142',
 'V144',
 'V148',
 'V153',
 'V155',
 'V157',
 'V168',
 'V174',
 'V179',
 'V181',
 'V183',
 'V185',
 'V186',
 'V190',
 'V191',
 'V192',
 'V193',
 'V194',
 'V196',
 'V198',
 'V199',
 'V211',
 'V218',
 'V230',
 'V232',
 'V235',
 'V236',
 'V237',
 'V240',
 'V241',
 'V248',
 'V250',
 'V252',
 'V254',
 'V255',
 'V260',
 'V269',
 'V281',
 'V284',
 'V286',
 'V290',
 'V293',
 'V295',
 'V296',
 'V297',
 'V298',
 'V299',
 'V300',
 'V301',
 'V302',
 'V305',
 'V309',
 'V311',
 'V316',
 'V318',
 'V319',
 'V320',
 'V321',
 'V325',
 'V327',
 'V328',
 'V330',
 'V334',
 'V337',
 'V339']

In [13]:
print('{} features are going to be dropped for being useless'.format(len(cols_to_drop)))

data = data.drop(cols_to_drop, axis=1)


135 features are going to be dropped for being useless


In [14]:
#Add NaN-count feature
data['NanTransactionCount'] = data.isna().sum(axis=1)
data_id['NanIdentityCount'] = data_id.isna().sum(axis=1)

In [15]:
# let's combine the data and work with the whole dataset
data = pd.merge(data, data_id, on='TransactionID', how='left')
del data_id

In [16]:
data['NanIdentityCount'] =  data['NanIdentityCount'].fillna(value=41)

In [17]:
save_features2(data,"data", ["NanIdentityCount"] ,suffix="NanIdentityCount")

In [18]:
data.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,NanIdentityCount
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.0
2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.0
2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.0
2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.0
2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,...,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M,17.0


In [19]:
data.columns

Index(['isFraud', 'TransactionDT', 'TransactionAmt', 'ProductCD', 'card1',
       'card2', 'card3', 'card4', 'card5', 'card6',
       ...
       'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo', 'NanIdentityCount'],
      dtype='object', length=300)

In [20]:
# Transforming Time
# https://www.kaggle.com/c/ieee-fraud-detection/discussion/100400#latest-579480
import datetime

START_DATE = '2017-12-01'
startdate = datetime.datetime.strptime(START_DATE, "%Y-%m-%d")

for dataset in (data,):
    dataset["Date"] = dataset['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds=x)))
    dataset['_Weekdays'] = dataset['Date'].dt.dayofweek
    dataset['_Hours'] = dataset['Date'].dt.hour
    dataset['_Days'] = dataset['Date'].dt.day

data = data.drop(['TransactionDT'],axis=1)
save_features2(data,'data', ["Date","_Weekdays","_Hours","_Days"], suffix="Dates")


In [21]:
# Transforming email domains

emails = {'gmail': 'google', 'att.net': 'att', 'twc.com': 'spectrum', 'scranton.edu': 'other', 'optonline.net': 'other', 'hotmail.co.uk': 'microsoft', 'comcast.net': 'other', 'yahoo.com.mx': 'yahoo', 'yahoo.fr': 'yahoo', 'yahoo.es': 'yahoo', 'charter.net': 'spectrum', 'live.com': 'microsoft', 'aim.com': 'aol', 'hotmail.de': 'microsoft', 'centurylink.net': 'centurylink', 'gmail.com': 'google', 'me.com': 'apple', 'earthlink.net': 'other', 'gmx.de': 'other', 'web.de': 'other', 'cfl.rr.com': 'other', 'hotmail.com': 'microsoft', 'protonmail.com': 'other', 'hotmail.fr': 'microsoft', 'windstream.net': 'other', 'outlook.es': 'microsoft', 'yahoo.co.jp': 'yahoo', 'yahoo.de': 'yahoo', 'servicios-ta.com': 'other', 'netzero.net': 'other', 'suddenlink.net': 'other', 'roadrunner.com': 'other', 'sc.rr.com': 'other', 'live.fr': 'microsoft', 'verizon.net': 'yahoo', 'msn.com': 'microsoft', 'q.com': 'centurylink', 'prodigy.net.mx': 'att', 'frontier.com': 'yahoo', 'anonymous.com': 'other', 'rocketmail.com': 'yahoo', 'sbcglobal.net': 'att', 'frontiernet.net': 'yahoo', 'ymail.com': 'yahoo', 'outlook.com': 'microsoft', 'mail.com': 'other', 'bellsouth.net': 'other', 'embarqmail.com': 'centurylink', 'cableone.net': 'other', 'hotmail.es': 'microsoft', 'mac.com': 'apple', 'yahoo.co.uk': 'yahoo', 'netzero.com': 'other', 'yahoo.com': 'yahoo', 'live.com.mx': 'microsoft', 'ptd.net': 'other', 'cox.net': 'other', 'aol.com': 'aol', 'juno.com': 'other', 'icloud.com': 'apple'}
us_emails = ['gmail', 'net', 'edu']

# train[['P_emaildomain_1', 'P_emaildomain_2', 'P_emaildomain_3']] = train['P_emaildomain'].str.split('.', expand=True)
# train[['R_emaildomain_1', 'R_emaildomain_2', 'R_emaildomain_3']] = train['R_emaildomain'].str.split('.', expand=True)
# test[['P_emaildomain_1', 'P_emaildomain_2', 'P_emaildomain_3']] = test['P_emaildomain'].str.split('.', expand=True)
# test[['R_emaildomain_1', 'R_emaildomain_2', 'R_emaildomain_3']] = test['R_emaildomain'].str.split('.', expand=True)

#https://www.kaggle.com/c/ieee-fraud-detection/discussion/100499#latest-579654
for c in ['P_emaildomain', 'R_emaildomain']:
    data[c + '_bin'] = data[c].map(emails)
    
    data[c + '_suffix'] = data[c].map(lambda x: str(x).split('.')[-1])
        
    data[c + '_suffix'] = data[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')
    
save_features2(data,'data', ["P_emaildomain_bin","P_emaildomain_suffix","R_emaildomain_bin","R_emaildomain_suffix"], suffix="Emails")

In [22]:
numerical_cols = ['id_%02d' % i for i in range(1,12)] + ["V%d"%i for i in range(1,340)] + ["D%d"%i for i in range(1,16)] + ["C%d"%i for i in range(1,15)]  + ['dist1','TransactionAmt', 'NanIdentityCount', 'NanTransactionCount', '_Weekdays', '_Hours', '_Days', 'Date', 'dist2']
one_hot_cols = ['M1', 'M2', 'M3','M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'card4', 'card6', 'ProductCD'] + ['id_%02d'%i for i in (12,15,16,28,29,32,34,35,36,37,38)]
label_cols = ['id_13', 'id_14', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27',
            'id_30', 'id_31',  'id_33', 'DeviceType', 'DeviceInfo', 'P_emaildomain',
            'R_emaildomain', 'card1', 'card2', 'card3',  'card5', 'addr1', 'addr2', 
            'P_emaildomain_bin', 'P_emaildomain_suffix', 'R_emaildomain_bin', 'R_emaildomain_suffix']
label_cols += one_hot_cols
# assert set(one_hot_cols).intersection(set(label_cols)) == set()
assert set(numerical_cols).intersection(set(label_cols)) == set()
assert set(numerical_cols).intersection(set(one_hot_cols)) == set()
set(numerical_cols + label_cols + one_hot_cols) - set( list(data.columns))
set( list(data.columns)) - set(numerical_cols + label_cols + one_hot_cols)
 

{'isFraud'}

In [25]:
json.dump(label_cols, open("pickles/categorial_features.json",'w'))

In [26]:
json.dump(numerical_cols, open("pickles/numerical_features.json",'w'))

In [23]:
save_features2(data,'data', data.columns, suffix="all_features")

0) TreValueErrorcategorial features accurately, find very diverse and unite them

1) Убрать коррелирующие фичи

2) Убрать ту из них что даёт меньший джини

3) TimeKFold instead of StratifiedKFold